In [2]:
import json

import polars as pl
from polars import col, lit

In [ ]:
with open("./data/playlists_UCdUkQwI7Ozo5KnTGMPhzuhw_20230826T182800.json", "r") as f:
    data = json.load(f)

In [34]:
videos = []

for playlist in data:
    videos.extend(playlist["items"])

In [ ]:
df = pl.DataFrame(videos)
df = df.filter(col("transcript").is_not_null())

In [ ]:
videos = []

for idx in range(len(df)):
    video = {
        "id": df[idx]["snippet"][0]["resourceId"]["videoId"],
        "title": df[idx]["snippet"][0]["title"],
        "published_at": datetime.strptime(df[idx]["snippet"][0]["publishedAt"], "%Y-%m-%dT%H:%M:%SZ"),
        "description": df[idx]["snippet"][0].get("description", None),
        "view_count": int(df[idx]["statistics"][0].get("viewCount", 0)),
        "like_count": int(df[idx]["statistics"][0].get("likeCount", 0)),
        "favorite_count": int(df[idx]["statistics"][0].get("favoriteCount", 0)),
        "transcript": " ".join([item["text"] for item in df[idx]["transcript"][0]]) if df[idx]["transcript"][0] is not None else None
    }
    
    videos.append(video)

In [38]:
df = pl.DataFrame(videos)
df.write_parquet("./data/videos.parquet")

In [ ]:
df = pl.read_parquet("./data/videos.parquet")
df

In [ ]:
df = df.unique(subset=["id"])

In [ ]:
df.group_by("id").len().filter(col("len") > 1).sort("len", descending=True)

In [2]:
from datetime import datetime

import chromadb
from chromadb.utils import embedding_functions

In [3]:
chromadb_client = chromadb.PersistentClient("./data/chromadb")

In [51]:
collection = chromadb_client.create_collection(
    name="samil_church_youtube", 
    embedding_function=embedding_functions.DefaultEmbeddingFunction(),
    metadata={
        "description": "samilchurch youtube videos",
        "created": str(datetime.now())
    }  
)

In [ ]:
collection = chromadb_client.get_collection("samil_church_youtube")
collection

In [ ]:
documents = []
metadatas = []
ids = []

for idx in range(len(df)):
    id = df[idx]["id"][0]
    transcript = df[idx]["transcript"][0]
    
    metadata = {
        "published_at": str(df[idx]["published_at"][0].timestamp()),
        "created_at": str(datetime.now().timestamp()),
    }
    
    documents.append(transcript)
    metadatas.append(metadata)
    ids.append(id)
    
collection.add(documents=documents, metadatas=metadatas, ids=ids)

In [ ]:
collection

In [ ]:
collection.query(query_texts=["이사야와 관련된 영상"], n_results=10)

In [ ]:
collection.count()